In [1]:
import pandas as pd

In [2]:
# read files: business, review, tip, user
business = pd.read_json("../data/business_city.json", lines = True)
review = pd.read_json("../data/review_city.json", lines = True)
tip = pd.read_json("../data/tip_city.json", lines = True)
user = pd.read_json("../data/user_city.json", lines = True)

In [3]:
business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,pQeaRpvuhoEqudo3uymHIQ,The Empanadas House,404 E Green St,Champaign,IL,61820,40.110446,-88.233073,4.5,5,1,"{'RestaurantsAttire': 'u'casual'', 'Restaurant...","Ethnic Food, Food Trucks, Specialty Food, Impo...","{'Monday': '11:30-14:30', 'Tuesday': '11:30-14..."
1,7Dv4_HAxsxvadEsT5fxQBg,Dependable Brakes & Exhaust,1110 Saw Mill Run Blvd,Pittsburgh,PA,15226,40.406667,-80.004450,5.0,5,1,{'BusinessAcceptsCreditCards': 'True'},"Automotive, Auto Repair","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
2,M_guz7Dj7hX0evS672wIwA,Chocolate Shoppe Ice Cream,2831 Parmenter St,Middleton,WI,53562,43.105310,-89.510142,3.5,6,1,"{'BikeParking': 'True', 'Caters': 'True', 'Whe...","Desserts, Food, Ice Cream & Frozen Yogurt","{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:0', ..."
3,03x6ZlJ7s39DHqfTJU1Ecg,Beechmont Country Club,29600 Chagrin Blvd,Beachwood,OH,44122,41.461448,-81.470127,4.5,7,1,"{'GoodForKids': 'True', 'BusinessAcceptsCredit...","Event Planning & Services, Venues & Event Spac...",None
4,_4Oe9V-qTpU5iemM9bphlA,Goodyear Auto Service Center,"800 Fort Duquesne Blvd, Ste 1",Pittsburgh,PA,15222,40.444614,-79.999344,3.5,12,1,{'BusinessAcceptsCreditCards': 'True'},"Oil Change Stations, Auto Repair, Automotive, ...","{'Monday': '7:0-18:0', 'Tuesday': '7:0-18:0', ..."


In [73]:
attr_dict = {"NoiseLevel": [], "BusinessParking": [], "BusinessAcceptsCreditCards": [], "RestaurantsPriceRange2": [], 
             "RestaurantsDelivery": [], "OutdoorSeating": [], "ByAppointmentOnly": [], "GoodForKids": []}
for ind in range(len(business)):
    attr = business.loc[ind, "attributes"]
    if attr == None: 
        for attribute in attr_dict.keys(): 
            attr_dict[attribute].append("NaN")
    else: 
        attr_key = attr.keys()
        for attribute in attr_dict.keys(): 
            if attribute in attr_key: 
                attr_dict[attribute].append(attr[attribute])
            else: 
                attr_dict[attribute].append("NaN")

In [87]:
attr_df = pd.DataFrame(attr_dict)
attr_df.insert(loc = 0, column = "stars", value = business["stars"])
attr_df

,stars,NoiseLevel,BusinessParking,BusinessAcceptsCreditCards,RestaurantsPriceRange2,RestaurantsDelivery,OutdoorSeating,ByAppointmentOnly,GoodForKids
0,4.5,u'quiet',None,True,1,True,False,False,False
1,5.0,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN
2,3.5,NaN,"{'garage': False, 'street': True, 'validated':...",True,1,NaN,NaN,NaN,NaN
3,4.5,NaN,NaN,False,NaN,NaN,NaN,NaN,True
4,3.5,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
36322,2.0,NaN,"{'garage': False, 'street': False, 'validated'...",True,2,NaN,NaN,NaN,NaN
36323,2.5,u'very_loud',"{'garage': False, 'street': False, 'validated'...",True,4,True,False,NaN,True
36324,4.0,NaN,NaN,True,NaN,False,False,False,True
36325,4.0,u'average',"{'garage': False, 'street': True, 'validated':...",True,2,False,True,NaN,False


In [88]:
attr_df.to_csv("../data/extract_attributes.csv", index = False)

In [76]:
sum(attr_df["NoiseLevel"] == "NaN")

27363

In [77]:
sum(attr_df["BusinessParking"] == "NaN")

14771

In [78]:
sum(attr_df["BusinessAcceptsCreditCards"] == "NaN")

7778

In [79]:
sum(attr_df["RestaurantsPriceRange2"] == "NaN")

15306

In [80]:
sum(attr_df["RestaurantsDelivery"] == "NaN")

24687

In [81]:
sum(attr_df["OutdoorSeating"] == "NaN")

24960

In [82]:
sum(attr_df["ByAppointmentOnly"] == "NaN")

28058

In [83]:
sum(attr_df["GoodForKids"] == "NaN")

23200